In [87]:
import pandas as pd
import numpy as np


data = pd.read_csv("2025 DCMP - Match Data.csv", low_memory = False)

data = data.dropna(subset=['ba_autoCoralCount'])

auto_cor_dat = data[['ba_autoCoralCount','a_L4','a_L3','a_L2','a_L1','match_number','team_key', 'r1','r2','r3','b1', 'b2', 'b3']]

print(auto_cor_dat)


     ba_autoCoralCount  a_L4  a_L3  a_L2  a_L1  match_number team_key  \
0                  7.0   2.0   0.0   0.0   0.0             1  frc1778   
1                  7.0   0.0   0.0   0.0   0.0             1  frc6831   
2                  7.0   2.0   2.0   0.0   0.0             1  frc2910   
3                  3.0   2.0   0.0   0.0   0.0             1   frc360   
4                  3.0   0.0   0.0   0.0   0.0             1  frc2412   
..                 ...   ...   ...   ...   ...           ...      ...   
577                3.0   0.0   0.0   0.0   0.0            96  frc4513   
578                3.0   2.0   0.0   0.0   0.0            96  frc3218   
579                4.0   0.0   0.0   0.0   0.0            96  frc6696   
580                4.0   0.0   0.0   0.0   0.0            96  frc9446   
581                4.0   2.0   0.0   0.0   0.0            96  frc1540   

          r1       r2       r3       b1       b2       b3  
0    frc1778  frc6831  frc2910   frc360  frc2412  frc9567  
1  

In [88]:
import pandas as pd
import numpy as np


wl_data = pd.read_csv("7034.csv", low_memory = False)
wl_data = wl_data.dropna(subset=["aL4"])

wl_auto_cor_dat = wl_data[['Match #','Robot','Team #','aL4','aL3','aL2','aL1']]

print(wl_auto_cor_dat)


     Match # Robot  Team #  aL4  aL3  aL2  aL1
0        1.0    R3  2910.0  1.0  2.0  0.0  0.0
1        1.0    R2  6831.0  2.0  0.0  0.0  0.0
2        1.0    R1  1778.0  1.0  0.0  0.0  0.0
3        1.0    B3  9567.0  0.0  0.0  0.0  0.0
4        1.0    B2  2412.0  0.0  0.0  0.0  0.0
..       ...   ...     ...  ...  ...  ...  ...
580    100.0    R2  3574.0  0.0  0.0  0.0  0.0
581    100.0    R1  4043.0  2.0  0.0  0.0  0.0
582    100.0    B3  2910.0  3.0  0.0  0.0  0.0
583    100.0    B2  9036.0  0.0  0.0  0.0  1.0
584    100.0    B1  9023.0  1.0  0.0  0.0  0.0

[585 rows x 7 columns]


In [89]:
# Convert `team_key` to a numeric column
data['Team #'] = data['team_key'].str.extract('(\d+)').astype(float)

# Merge WL_data with data based on 'Match #' and 'Team #'
wl_auto_cor_dat = wl_auto_cor_dat.merge(
    data[['ba_autoCoralCount', 'match_number', 'Team #']],
    left_on=['Match #', 'Team #'],
    right_on=['match_number', 'Team #'],
    how='left'
)
wl_auto_cor_dat = wl_auto_cor_dat.dropna(subset=["ba_autoCoralCount"])
# Drop the redundant `match_number` column from the merge
wl_auto_cor_dat = wl_auto_cor_dat.drop(columns=['match_number'])

# Print the resulting DataFrame
print(wl_auto_cor_dat)

     Match # Robot  Team #  aL4  aL3  aL2  aL1  ba_autoCoralCount
0        1.0    R3  2910.0  1.0  2.0  0.0  0.0                7.0
1        1.0    R2  6831.0  2.0  0.0  0.0  0.0                7.0
2        1.0    R1  1778.0  1.0  0.0  0.0  0.0                7.0
3        1.0    B3  9567.0  0.0  0.0  0.0  0.0                3.0
4        1.0    B2  2412.0  0.0  0.0  0.0  0.0                3.0
..       ...   ...     ...  ...  ...  ...  ...                ...
556     96.0    R2  4513.0  0.0  0.0  0.0  0.0                3.0
557     96.0    R1   948.0  1.0  0.0  0.0  0.0                3.0
558     96.0    B3  1540.0  2.0  0.0  0.0  0.0                4.0
559     96.0    B2  9446.0  0.0  0.0  0.0  0.0                4.0
560     96.0    B1  6696.0  2.0  0.0  0.0  0.0                4.0

[557 rows x 8 columns]


<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
C:\Users\aarus\AppData\Local\Temp\ipykernel_11980\2518765814.py:2: SyntaxWarning: invalid escape sequence '\d'
  data['Team #'] = data['team_key'].str.extract('(\d+)').astype(float)


In [90]:
# Add an `Alliance` column based on the `Robot` column
wl_auto_cor_dat['Alliance'] = wl_auto_cor_dat['Robot'].apply(lambda x: 'Red' if 'R' in x else 'Blue')

# Group by `Match #` and `Alliance` to calculate the sum of `aL4`, `aL3`, `aL2`, and `aL1`
alliance_sums = wl_auto_cor_dat.groupby(['Match #', 'Alliance'])[['aL4', 'aL3', 'aL2', 'aL1']].sum()

# Add a new column `alliance_sum` for the total sum across `aL4`, `aL3`, `aL2`, and `aL1`
alliance_sums['alliance_sum'] = alliance_sums.sum(axis=1)

# Reset the index to make it easier to merge back
alliance_sums = alliance_sums.reset_index()

# Merge the alliance sums back into the original DataFrame
wl_auto_cor_dat = wl_auto_cor_dat.merge(
    alliance_sums[['Match #', 'Alliance', 'alliance_sum']],
    on=['Match #', 'Alliance'],
    how='left'
)

wl_auto_cor_dat['alliance_delta'] = abs(wl_auto_cor_dat['ba_autoCoralCount'] - wl_auto_cor_dat['alliance_sum'])
# Print the updated DataFrame
print(wl_auto_cor_dat)

     Match # Robot  Team #  aL4  aL3  aL2  aL1  ba_autoCoralCount Alliance  \
0        1.0    R3  2910.0  1.0  2.0  0.0  0.0                7.0      Red   
1        1.0    R2  6831.0  2.0  0.0  0.0  0.0                7.0      Red   
2        1.0    R1  1778.0  1.0  0.0  0.0  0.0                7.0      Red   
3        1.0    B3  9567.0  0.0  0.0  0.0  0.0                3.0     Blue   
4        1.0    B2  2412.0  0.0  0.0  0.0  0.0                3.0     Blue   
..       ...   ...     ...  ...  ...  ...  ...                ...      ...   
552     96.0    R2  4513.0  0.0  0.0  0.0  0.0                3.0      Red   
553     96.0    R1   948.0  1.0  0.0  0.0  0.0                3.0      Red   
554     96.0    B3  1540.0  2.0  0.0  0.0  0.0                4.0     Blue   
555     96.0    B2  9446.0  0.0  0.0  0.0  0.0                4.0     Blue   
556     96.0    B1  6696.0  2.0  0.0  0.0  0.0                4.0     Blue   

     alliance_sum  alliance_delta  
0             6.0          

In [91]:
# Step 1: Assign alliance based on `team_key` membership in `r1`, `r2`, `r3` or `b1`, `b2`, `b3`
def assign_alliance(row):
    if row['team_key'] in [row['r1'], row['r2'], row['r3']]:
        return 'Red'
    elif row['team_key'] in [row['b1'], row['b2'], row['b3']]:
        return 'Blue'
    return None

auto_cor_dat['Alliance'] = auto_cor_dat.apply(assign_alliance, axis=1)

# Step 2: Compute `alliance_sum` by grouping by `match_number` and `Alliance`
alliance_sums = auto_cor_dat.groupby(['match_number', 'Alliance'])[['a_L4', 'a_L3', 'a_L2', 'a_L1']].sum()

# Add a new column `alliance_sum` that contains the total sum
alliance_sums['alliance_sum'] = alliance_sums.sum(axis=1)

# Reset index for merging
alliance_sums = alliance_sums.reset_index()

# Step 3: Merge the alliance sums back into the original DataFrame
auto_cor_dat = auto_cor_dat.merge(
    alliance_sums[['match_number', 'Alliance', 'alliance_sum']],
    on=['match_number', 'Alliance'],
    how='left'
)

# Step 4: Calculate `alliance_delta` as the absolute difference between `ba_autoCoralCount` and `alliance_sum`
auto_cor_dat['alliance_delta'] = abs(auto_cor_dat['ba_autoCoralCount'] - auto_cor_dat['alliance_sum'])

# Print the updated DataFrame
print(auto_cor_dat)

     ba_autoCoralCount  a_L4  a_L3  a_L2  a_L1  match_number team_key  \
0                  7.0   2.0   0.0   0.0   0.0             1  frc1778   
1                  7.0   0.0   0.0   0.0   0.0             1  frc6831   
2                  7.0   2.0   2.0   0.0   0.0             1  frc2910   
3                  3.0   2.0   0.0   0.0   0.0             1   frc360   
4                  3.0   0.0   0.0   0.0   0.0             1  frc2412   
..                 ...   ...   ...   ...   ...           ...      ...   
571                3.0   0.0   0.0   0.0   0.0            96  frc4513   
572                3.0   2.0   0.0   0.0   0.0            96  frc3218   
573                4.0   0.0   0.0   0.0   0.0            96  frc6696   
574                4.0   0.0   0.0   0.0   0.0            96  frc9446   
575                4.0   2.0   0.0   0.0   0.0            96  frc1540   

          r1       r2       r3       b1       b2       b3 Alliance  \
0    frc1778  frc6831  frc2910   frc360  frc2412  frc

C:\Users\aarus\AppData\Local\Temp\ipykernel_11980\393549060.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  auto_cor_dat['Alliance'] = auto_cor_dat.apply(assign_alliance, axis=1)


In [92]:
print(wl_auto_cor_dat)

     Match # Robot  Team #  aL4  aL3  aL2  aL1  ba_autoCoralCount Alliance  \
0        1.0    R3  2910.0  1.0  2.0  0.0  0.0                7.0      Red   
1        1.0    R2  6831.0  2.0  0.0  0.0  0.0                7.0      Red   
2        1.0    R1  1778.0  1.0  0.0  0.0  0.0                7.0      Red   
3        1.0    B3  9567.0  0.0  0.0  0.0  0.0                3.0     Blue   
4        1.0    B2  2412.0  0.0  0.0  0.0  0.0                3.0     Blue   
..       ...   ...     ...  ...  ...  ...  ...                ...      ...   
552     96.0    R2  4513.0  0.0  0.0  0.0  0.0                3.0      Red   
553     96.0    R1   948.0  1.0  0.0  0.0  0.0                3.0      Red   
554     96.0    B3  1540.0  2.0  0.0  0.0  0.0                4.0     Blue   
555     96.0    B2  9446.0  0.0  0.0  0.0  0.0                4.0     Blue   
556     96.0    B1  6696.0  2.0  0.0  0.0  0.0                4.0     Blue   

     alliance_sum  alliance_delta  
0             6.0          

In [93]:
import pandas as pd

# Assuming `data` and `wl_auto_cor_dat` are already defined and contain the required data

# Step 1: Ensure both DataFrames are aligned
wl_auto_cor_dat['Team #'] = wl_auto_cor_dat['Team #'].astype(int).astype(str)  # Convert to string for matching
auto_cor_dat['team_key'] = auto_cor_dat['team_key'].str.replace('frc', '')  # Remove 'frc' prefix for alignment

# Rename columns in wl_auto_cor_dat to align with `data` (optional)
wl_auto_cor_dat = wl_auto_cor_dat.rename(columns={
    'Match #': 'match_number',
    'Team #': 'team_key',
    'aL4': 'a_L4',
    'aL3': 'a_L3',
    'aL2': 'a_L2',
    'aL1': 'a_L1'
})

# Step 2: Merge the two DataFrames on `match_number` and `team_key`
# Use an outer merge to preserve all rows
merged = pd.merge(
    auto_cor_dat,
    wl_auto_cor_dat,
    on=['match_number', 'team_key'],
    suffixes=('_data', '_wl'),
    how='outer'  # Use 'outer' to ensure no rows are lost
)

# Step 3: Handle missing values in `alliance_delta` columns
merged['alliance_delta_data'] = abs(merged['ba_autoCoralCount_data'] - merged['alliance_sum_data']).fillna(9999)
merged['alliance_delta_wl'] = abs(merged['ba_autoCoralCount_wl'] - merged['alliance_sum_wl']).fillna(9999)

# Step 4: Calculate `min_delta` for each match_number and team_key
merged['min_delta'] = merged[['alliance_delta_data', 'alliance_delta_wl']].min(axis=1)

# Step 5: Select rows with the minimum delta
# Ensure only one row per match_number and team_key
fixed_auto = merged.loc[
    (merged['alliance_delta_data'] == merged['min_delta']) |
    (merged['alliance_delta_wl'] == merged['min_delta'])
]

# Step 6: Assign an `order` column to explicitly group R1, R2, R3, B1, B2, B3
def assign_order(row):
    if row['team_key'] == row['r1']:
        return 'R1'
    elif row['team_key'] == row['r2']:
        return 'R2'
    elif row['team_key'] == row['r3']:
        return 'R3'
    elif row['team_key'] == row['b1']:
        return 'B1'
    elif row['team_key'] == row['b2']:
        return 'B2'
    elif row['team_key'] == row['b3']:
        return 'B3'
    return None

fixed_auto['order'] = fixed_auto.apply(assign_order, axis=1)

# Step 7: Sort the DataFrame by `match_number` and `order`
# Use a predefined order for sorting
order_sorting = {'R1': 1, 'R2': 2, 'R3': 3, 'B1': 4, 'B2': 5, 'B3': 6}
fixed_auto['order_sort'] = fixed_auto['order'].map(order_sorting)  # Map the order for sorting
fixed_auto = fixed_auto.sort_values(by=['match_number', 'order_sort']).drop(columns=['order_sort'])

# Step 8: Clean up the `fixed_auto` DataFrame for output
fixed_auto = fixed_auto[[
    'match_number', 'team_key', 'ba_autoCoralCount_data',
    'a_L4_data', 'a_L3_data', 'a_L2_data', 'a_L1_data',
    'a_L4_wl', 'a_L3_wl', 'a_L2_wl', 'a_L1_wl', 'min_delta'
]]

# Step 9: Save the resulting DataFrame to a CSV file
fixed_auto.to_csv('auto_fixed.csv', index=False)

# Print a confirmation message
print("The fixed_auto DataFrame has been saved to 'auto_fixed.csv'.")


The fixed_auto DataFrame has been saved to 'auto_fixed.csv'.
